In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations 
from collections import defaultdict
from scipy import stats

In [ ]:
!pip install pandas numpy matplotlib seaborn scipy openpyxl

In [ ]:
INPUT_FILE = "consistency_analysis.xlsx"

**DATA DICTIONARY**
|Feature|Type|Description|
| --- | --- | --- |
|Contract|string|Contract identifier (e.g., "2023-03660-000")|
|Difficulty|string|Contract complexity: "Easy", "Medium", or "Hard"|
|Service_Raw|string|Role or service title (e.g., "Mechanic", "Police")|
|Coder|string|Coder: "W", "G", or "D"|
|Round|int|Exercise number: 1 (employees) or 2 (services)|
|NAICS_Raw|string|Assigned NAICS code, semicolon-delimited if multiple|

In [ ]:
def load_data(filepath):
    """
    Loads data into pandas
    
    :param filepath: filepath of xlsx
    """
    df = pd.read_excel(filepath)
    return df

df = load_data(INPUT_FILE)

In [ ]:
def eda(df):
    """
    Performs exploratory data analysis
    
    :param df: pandas dataframe for EDA
    """

    print("=" * 60)
    print("EXPLORATORY DATA ANALYSIS")
    print("=" * 60)

    print(f"\nShape: {df.shape[0]} rows × {df.shape[1]} columns")

    print(f"\nColumns: {df.columns.tolist()}")

    print(f"\nData types:\n{df.dtypes}")

    print(f"\nUnique contracts: {df['Contract'].nunique()}")
    print(f"Unique coders: {df['Coder'].unique().tolist()}")
    print(f"Rounds: {df['Round'].unique().tolist()}")
    print(f"Difficulty levels: {df['Difficulty'].unique().tolist()}")

    print(f"\nMissing values:\n{df.isnull().sum()}")

    print(f"\nFirst 10 rows:\n{df.head(10).to_string()}")

eda(df)

In [ ]:
def clean(df):
    """
    Clean dataframe
    
    :param df: pandas dataframe for cleaning
    """
    df_clean = df.copy()

    df_clean['Contract'] = df_clean['Contract'].str.strip()

    df_clean['has_naics'] = df_clean['NAICS_Raw'].notna() & (df_clean['NAICS_Raw'] != '')

    df_clean['is_multicode'] = df_clean['NAICS_Raw'].str.contains(';', na=False)

    df_clean['lookup_key'] = df_clean['Contract'] + '|' + df_clean['Service_Raw']

    print(f"Cleaned data: {len(df_clean)} rows (from {len(df)} original)")
    print(f"Rows with valid NAICS: {df_clean['has_naics'].sum()}")
    print(f"Multi-code entries: {df_clean['is_multicode'].sum()}")
    
    return df_clean

df = clean(df)

In [ ]:
def get_codes(naics_value):
    """
    Flexible parsing for NAICS code string to convert to set depending on number of values
    
    :param naics_value: NAICS string from a given row
    """

    if pd.isna(naics_value) or naics_value == "":
        return set()
    
    codes = str(naics_value).split(';')
    return set(code.strip() for code in codes if code.strip())

**CONTRACT-LEVEL ANALYSIS**

Jaccard index measures similarity between two sets:

$J(A, B) = \frac{|A \cap B|}{|A \cup B|}$ 

Given set A of codes assigned to a given contract by one coder and set B of codes assigned to a given contract by a different coder

- J = 1.0: Identical sets (perfect agreement on coding for a given contract)
- J = 0.5: Half overlap
- J = 0.0: No overlap (complete disagreement)

Jaccard compares sets of codes instead of item-by-item matches which is critical because *coders identified different numbers of services* for each contract

In [ ]:
def jaccard_similarity(set1, set2):
    """
    Calculate Jaccard similarity between two sets
    
    :param set1: First set of codes
    :param set2: Second set of codes
    """

    # Handle empty sets
    if len(set1) == 0 and len(set2) == 0:
        return 1.0  # Both empty = perfect agreement
    
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    
    if union == 0:
        return 0.0
    
    return intersection / union

In [ ]:
def get_coder_code_sets(df, contract):
    """
    For a given contract, get the set of all NAICS codes assigned by each coder.

    :param df: cleaned pandas dataframe
    :param contract: Contract ID (str)
    """
    contract_df = df[df['Contract'] == contract]
    
    coder_codes = {}
    for coder in ['W', 'G', 'D']:
        coder_df = contract_df[contract_df['Coder'] == coder]
        
        if len(coder_df) == 0:
            continue
        
        # Aggregate all codes assigned by this coder for this contract
        all_codes = set()
        for naics_val in coder_df['NAICS_Raw'].dropna():
            all_codes.update(get_codes(naics_val))
        
        if all_codes:  # Only include if coder assigned at least one code
            coder_codes[coder] = all_codes
    
    return coder_codes

In [ ]:
def calculate_contract_jaccard(df):
    """
    Calculate pairwise Jaccard similarity for all contracts.
    
    :param df: cleaned pandas dataframe
    """
    results = []
    
    for contract in df['Contract'].unique():
        # Get difficulty for this contract
        difficulty = df[df['Contract'] == contract]['Difficulty'].iloc[0]
        
        # Get code sets for each coder
        coder_codes = get_coder_code_sets(df, contract)
        
        # Calculate pairwise Jaccard for all coder pairs
        coders_present = list(coder_codes.keys())
        
        for coder1, coder2 in combinations(coders_present, 2):
            set1 = coder_codes[coder1]
            set2 = coder_codes[coder2]
            
            intersection = set1 & set2
            union = set1 | set2
            jaccard = jaccard_similarity(set1, set2)
            
            results.append({
                'Contract': contract,
                'Difficulty': difficulty,
                'Coder1': coder1,
                'Coder2': coder2,
                'Pair': f"{coder1}-{coder2}",
                'Codes_1': len(set1),
                'Codes_2': len(set2),
                'Intersection': len(intersection),
                'Union': len(union),
                'Jaccard': jaccard,
                'Shared_Codes': ';'.join(sorted(intersection)) if intersection else '',
                'Unique_to_1': ';'.join(sorted(set1 - set2)) if (set1 - set2) else '',
                'Unique_to_2': ';'.join(sorted(set2 - set1)) if (set2 - set1) else ''
            })
    
    return pd.DataFrame(results)

jaccard_df = calculate_contract_jaccard(df)

In [ ]:
def summarize_jaccard_by_difficulty(jaccard_df):
    """
    Calculate mean Jaccard similarity by difficulty level.

    :param jaccard_df: Jaccard results
    """
    summary = jaccard_df.groupby('Difficulty')['Jaccard'].agg([
        ('Mean', 'mean'),
        ('Std', 'std'),
        ('Min', 'min'),
        ('Max', 'max'),
        ('N_Pairs', 'count')
    ]).round(3)
    
    # Reorder by difficulty
    difficulty_order = ['Easy', 'Medium', 'Hard']
    summary = summary.reindex(difficulty_order)
    
    print("=" * 60)
    print("CONTRACT-LEVEL JACCARD SIMILARITY BY DIFFICULTY")
    print("=" * 60)
    print(summary.to_string())
    
    return summary

jaccard_summary = summarize_jaccard_by_difficulty(jaccard_df)